# Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

tf.random.set_seed(7)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

2022-11-21 03:27:45.184123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read Data and Pruning

In [2]:
def prune_columns(dfm:pd.DataFrame, cols:list):
    return dfm.drop(cols, axis=1)

In [3]:
def read_data(ddir:str):
    dataframe = pd.read_csv(ddir)
    pruned_dataframe = prune_columns(dataframe, ['ImagesName'])
    return pruned_dataframe

# Split and Scale Data (Standard Scaler)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
def split_data(dataframe:pd.DataFrame):
    X = dataframe.drop(['Labels'], axis=1)
    y = dataframe['Labels']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

# Build Model

In [6]:
def optimize(models: list,
             X_train: np.array,
             y_train: np.array,
             X_test: np.array,
             y_test: np.array,
             epochs: int = 50,
             verbose: int = 0) -> pd.DataFrame:
    
    # We'll store the results here
    results = []
    
    def train(model: tf.keras.Sequential) -> dict:
        # Change this however you want 
        # We're not optimizing this part today
        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[
                tf.keras.metrics.BinaryAccuracy(name='accuracy')
            ]
        )
        
        # Train the model
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            verbose=verbose
        )
        
        
        # Make predictions on the test set
        preds = model.predict(X_test)
        prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(preds)]
        
        model.evaluate(X_test, y_test, verbose=0)
        model.save(f'/Users/teguhsatya/Desktop/helloNew/glokumarch/modelling/MLP/MODEL_HISTORY4/nn_{model.name}.h5')
        # Return evaluation metrics on the test set
        return {
            'model_name': model.name,
            'test_accuracy': accuracy_score(y_test, prediction_classes),
            'test_precision': precision_score(y_test, prediction_classes),
            'test_recall': recall_score(y_test, prediction_classes),
            'test_f1': f1_score(y_test, prediction_classes)
        }
    
    # Train every model and save results
    for model in models:
        try:
            print(model.name, end=' ... ')
            res = train(model=model)
            results.append(res)
        except Exception as e:
            print(f'{model.name} --> {str(e)}')
        
    return pd.DataFrame(results)

In [7]:
def build_models(num_layers: int, min_nodes_per_layer: int,\
                 max_nodes_per_layer: int, node_step_size: int,\
                 input_shape: tuple, hidden_layer_activation: str = 'relu',\
                 num_nodes_at_output: int = 1, output_layer_activation: str = 'sigmoid') -> list:
    
    node_options = list(range(min_nodes_per_layer, max_nodes_per_layer + 1, node_step_size))
    layer_possibilities = [node_options] * num_layers
    layer_node_permutations = list(itertools.product(*layer_possibilities))
    
    models = []
    for permutation in layer_node_permutations:
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
        model_name = ''
        
        for nodes_at_layer in permutation:
            model.add(tf.keras.layers.Dense(nodes_at_layer, activation=hidden_layer_activation))
            model_name += f'dense{nodes_at_layer}_'
        
        model.add(tf.keras.layers.Dense(num_nodes_at_output, activation=output_layer_activation))
        model._name = model_name[:-1]
        models.append(model)
        
    return models

# Evaluation

# Main

In [8]:
# features_data_dir = '/Users/teguhsatya/Desktop/helloNew/glokumarch/exportedDataframe/matchedFeatures.csv'
features_data_dir = '/Users/teguhsatya/Dev/segeralulus/exportedDataframe/a_features.csv'

In [9]:
df = read_data(features_data_dir)

In [10]:
df.sample(5)

,CDR,VCDR,RDR,I,S,N,T,Labels
119,0.547753,0.528455,0.084270,140,154,93,138,1
144,0.418033,0.467890,0.172131,225,201,246,212,1
114,0.487500,0.440181,0.097500,161,222,160,150,1
40,0.436170,0.437100,0.190000,237,234,227,226,0
64,0.349845,0.352025,0.263158,126,168,137,129,0


In [11]:
X_train_scaled, X_test_scaled, y_train, y_test = split_data(df)

In [12]:
all_models = build_models(\
                          num_layers=3,
                          min_nodes_per_layer=5,
                          max_nodes_per_layer=25,
                          node_step_size=5,
                          input_shape=(7,)\
                         )

In [13]:
optimization_result = optimize(\
                               models=all_models,\
                               X_train=X_train_scaled,\
                               y_train=y_train,\
                               X_test=X_test_scaled,\
                               y_test=y_test,
                               epochs=30
                              )

2/2 [==============================] - 0s 2ms/step
dense5_dense5_dense25 ... WARNING:tensorflow:5 out of the last 9 calls to <function Model.make_predict_function.<locals>.predict_function at 0x12e920550> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 [==============================] - 0s 2ms/step
dense5_dense10_dense5 ... WARNING:tensorflow:6 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x12e922830> triggered tf.function 

In [14]:
optimization_result.sort_values(by='test_accuracy', ascending=False)

,model_name,test_accuracy,test_precision,test_recall,test_f1
124,dense25_dense25_dense25,0.852941,0.800000,0.857143,0.827586
40,dense10_dense20_dense5,0.852941,0.909091,0.714286,0.800000
123,dense25_dense25_dense20,0.852941,0.764706,0.928571,0.838710
120,dense25_dense25_dense5,0.852941,0.800000,0.857143,0.827586
117,dense25_dense20_dense15,0.852941,0.800000,0.857143,0.827586
...,...,...,...,...,...
15,dense5_dense20_dense5,0.558824,0.476190,0.714286,0.571429
0,dense5_dense5_dense5,0.558824,0.473684,0.642857,0.545455
10,dense5_dense15_dense5,0.411765,0.400000,0.857143,0.545455
4,dense5_dense5_dense25,0.411765,0.411765,1.000000,0.583333


In [15]:
optimization_result.to_csv("/Users/teguhsatya/Desktop/helloNew/glokumarch/architecture/architectureComparison/m4Layers_comparisonResult.csv",index=False)